BÀI 9 : THUẬT TÁN ENSEMBLE LEARNING

BÀI 9A. PHÂN LOẠI VỚI THUẬT TOÁN BAGGING CLASSIFIER

VÍ DỤ 1: PHÂN LOẠI CHO VAY

In [1]:
# import thư viện
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(7,3), 'figure.dpi':120})

In [2]:
#Link data phần 9a
link='https://drive.google.com/file/d/1ygyPd9sTlZx361rUYeNtd4WcIPz6Pq-7/view?usp=sharing'
#Đọc data
path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]
df = pd.read_csv(path)

DATA GỒM 13 CỘT

Loan_ID

Gender

Married

Dependents

Education

Self_Employed

ApplicantIncome

CoapplicantIncome

LoanAmount

Loan_Amount_Term

Credit_History

Property_Area

Loan_Status (y)



In [3]:
# xem thông tin cơ bản của dữ liệu
df.tail(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [4]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [5]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
# có một số dữ liệu là NaN mà chúng ta cần xử lí

df.dropna(inplace=True)

In [7]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [8]:
df.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:
# bảng dữ liệu dạng chuỗi nên chúng ta cần chjyeenr sang dạng số để máy tính có thể hiểu và tính toán
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder # thư viện max hóa các string
df = df.apply(LabelEncoder().fit_transform) # mã hóa dữ liệu (không mã hóa dữ liệu có tính chất liên tục)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,0,1,1,1,0,0,247,50,76,7,1,0,0
2,1,1,1,0,0,1,112,0,23,7,1,2,1
3,2,1,1,0,1,0,74,135,68,7,1,2,1
4,3,1,0,0,0,0,305,0,89,7,1,2,1
5,4,1,1,2,0,1,281,196,159,7,1,2,1


In [10]:
# chuẩn bị data đẫ xong ta phân chia các tập train/test
from sklearn.model_selection import train_test_split
y = df['Loan_Status'].values
x = df.iloc[:,1:-1].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [11]:
# Tính bằng phương pháp Logistic Regression

from sklearn.linear_model import LogisticRegression
logis = LogisticRegression()
logis.fit(X_train, y_train)
y_pred = logis.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.41      0.57        51
           1       0.75      0.98      0.85        93

    accuracy                           0.78       144
   macro avg       0.83      0.70      0.71       144
weighted avg       0.81      0.78      0.75       144



c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# tính bằng phương pháp BaggingClassifier
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(logis)
bag.fit(X_train, y_train)
accuracy_bag = bag.score(X_test, y_test)
y_pred = bag.predict(X_test)


c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

In [13]:
print('Accuracy: {:.2f}'.format(accuracy_bag))

Accuracy: 0.78


In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.43      0.59        51
           1       0.76      0.98      0.85        93

    accuracy                           0.78       144
   macro avg       0.84      0.70      0.72       144
weighted avg       0.81      0.78      0.76       144



In [15]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=8)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8)

In [16]:
y_pred = dt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.49      0.54        51
           1       0.75      0.82      0.78        93

    accuracy                           0.70       144
   macro avg       0.67      0.65      0.66       144
weighted avg       0.69      0.70      0.69       144



PHÂN LOẠI VỚI THUẬT TOÁN ADA BOOST

VÍ DỤ 2: TEST THUẬT TOÁN ADA BOOST VỚI DỮ LIỆU ƠR TRÊN

In [17]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=10)
ada.fit(X_train, y_train)
y_ped = ada.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.49      0.54        51
           1       0.75      0.82      0.78        93

    accuracy                           0.70       144
   macro avg       0.67      0.65      0.66       144
weighted avg       0.69      0.70      0.69       144



VÍ DỤ 4: TEST THUẬT TOÁN XGBoost VỚI DỮ LIỆU Ở PHẦN 9A

In [18]:
import xgboost as xgb
Xg = xgb.XGBClassifier(max_depth=3)
Xg.fit(X_train, y_train)
accuracy_xg = Xg.score(X_test, y_test)
print("Accuracy: {:.2f}".format(accuracy_xg))

Accuracy: 0.75


In [21]:
y_pred = Xg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.55      0.61        51
           1       0.78      0.86      0.82        93

    accuracy                           0.75       144
   macro avg       0.73      0.70      0.71       144
weighted avg       0.74      0.75      0.74       144



VÍ DỤ 5: TEST THUẬT TÁN LIGHT GBM VỚI DỮ LIỆU Ở PHẦN 9A

In [23]:
import lightgbm as lgb
Lig = lgb.LGBMClassifier()
Lig.fit(X_train, y_train)
accuracy_lig = Lig.score(X_test, y_test)
print('Accuracy: {:.2f}'.format(accuracy_lig))


Accuracy: 0.72


In [24]:
y_pred = Lig.predict(X_test)
print(classification_report(y_test, y_ped))

              precision    recall  f1-score   support

           0       0.86      0.47      0.61        51
           1       0.77      0.96      0.85        93

    accuracy                           0.78       144
   macro avg       0.81      0.71      0.73       144
weighted avg       0.80      0.78      0.77       144



In [25]:
# tìm kieems siêu tham số tối ưu cho mô hình

from sklearn.model_selection import GridSearchCV
hps = {
    'num_leaves' : [1, 10, 100, 1000],
    'learning_rate' : [0.00001, 0.0001, 0.001, 0.01, 1],
    'n_estimators' : [1, 10, 100, 1000],
}
grids = GridSearchCV(Lig, hps)

In [26]:
grids.fit(X_train, y_train)

c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Boost

GridSearchCV(estimator=LGBMClassifier(),
             param_grid={'learning_rate': [1e-05, 0.0001, 0.001, 0.01, 1],
                         'n_estimators': [1, 10, 100, 1000],
                         'num_leaves': [1, 10, 100, 1000]})

In [27]:
print('Accuracy: {:.2f}'.format(grids.best_score_))

Accuracy: 0.82


In [28]:
y_pred = grids.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.41      0.57        51
           1       0.75      0.98      0.85        93

    accuracy                           0.78       144
   macro avg       0.83      0.70      0.71       144
weighted avg       0.81      0.78      0.75       144



In [29]:
from sklearn.model_selection import RandomizedSearchCV
hps = {
    'num_leaves' : [1, 10, 100, 1000],
    'learning_rate' : [0.00001, 0.0001, 0.001, 0.01, 1],
    'n_estimators' : [1, 10, 100, 1000],
}
Random = RandomizedSearchCV(Lig, hps)


In [30]:
Random.fit(X_train, y_train)

c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "c:\Users\Quynh Nhu\anaconda3\Lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster 

RandomizedSearchCV(estimator=LGBMClassifier(),
                   param_distributions={'learning_rate': [1e-05, 0.0001, 0.001,
                                                          0.01, 1],
                                        'n_estimators': [1, 10, 100, 1000],
                                        'num_leaves': [1, 10, 100, 1000]})

In [31]:
y_pred = Random.predict(X_test)
print(classification_report(y_test, y_ped))

              precision    recall  f1-score   support

           0       0.86      0.47      0.61        51
           1       0.77      0.96      0.85        93

    accuracy                           0.78       144
   macro avg       0.81      0.71      0.73       144
weighted avg       0.80      0.78      0.77       144

